In [1]:
import pandas as pd

# COVID-19

In [2]:
# Total cases

In [12]:
    # World regions
world_total = pd.read_csv('output_data/time_series/total-world-covid19.csv')
world_total['Date_Confirmed'] = pd.DatetimeIndex(world_total['Date_Confirmed'])
world_total = world_total[['Date_Confirmed','China','Italy','Spain','United Kingdom','Korea, South','US','Mexico']]

    # Mexico regions
mexico_total = pd.read_csv('output_data/time_series/total-mexico-covid19.csv')
mexico_total['Date_Confirmed'] = pd.DatetimeIndex(mexico_total['Date_Confirmed'])
mexico_total['Mexico_SS'] = mexico_total.sum(axis=1) # Sum rows for Mexico total (to indicate: Secretaría de Salud)

In [13]:
# Daily cases

In [14]:
    # World regions
world_daily = pd.read_csv('output_data/time_series/daily-world_covid19.csv')
world_daily['Date_Confirmed'] = pd.DatetimeIndex(world_daily['Date_Confirmed'])
world_daily = world_daily[['Date_Confirmed','China','Italy','Spain','United Kingdom','Korea, South','US','Mexico']]

    # Mexico regions
mexico_daily = pd.read_csv('output_data/time_series/daily-mexico-covid19.csv')
mexico_daily['Date_Confirmed'] = pd.DatetimeIndex(mexico_daily['Date_Confirmed'])
mexico_daily['Mexico_SS'] = mexico_daily.sum(axis=1) # Sum rows for Mexico total (to indicate: Secretaría de Salud)

In [15]:
# Filter time series

In [16]:
    # Total
world_total = world_total.filter_date('Date_Confirmed','2020-02-15','2020-04-18')
mexico_total = mexico_total.filter_date('Date_Confirmed','2020-02-15','2020-04-18')

    # Daily
world_daily = world_daily.filter_date('Date_Confirmed','2020-02-15','2020-04-18')
mexico_daily = mexico_daily.filter_date('Date_Confirmed','2020-02-15','2020-04-18')

In [ ]:
# Merge world and Mexico data and produce time series

In [18]:
import numpy as np

In [23]:
    # Total
total = world_total.merge(mexico_ts, on='Date_Confirmed', how='outer')
total = total.fillna(method='ffill') # First fill missing time series totals with previous row value
total = total.replace(0, np.nan) # Now convert 0 to NaN

    # Daily
daily = world_daily.merge(mexico_daily, on='Date_Confirmed', how='outer')

In [24]:
# Save

In [25]:
total.to_csv('output_data/time_series/total.csv', index=False)
daily.to_csv('output_data/time_series/daily.csv', index=False)

In [166]:
# Make sure datetimes are correct

In [27]:
    # Total
total['Date_Confirmed'] = pd.to_datetime(total.Date_Confirmed, format='%d-%m-%Y')
total['Date_Confirmed'] = total['Date_Confirmed'].dt.strftime('%d-%m-%Y')
total = total.set_index(pd.DatetimeIndex(total['Date_Confirmed']))
total.index = total.index.strftime('%d-%m-%Y')

    # Daily
daily['Date_Confirmed'] = pd.to_datetime(daily.Date_Confirmed, format='%d-%m-%Y')
daily['Date_Confirmed'] = daily['Date_Confirmed'].dt.strftime('%d-%m-%Y')
daily = daily.set_index(pd.DatetimeIndex(daily['Date_Confirmed']))
daily.index = daily.index.strftime('%d-%m-%Y')

In [51]:
# Melt to long format

In [31]:
tbl_total = pd.DataFrame(total.set_index(['Date_Confirmed']).rename_axis(['Region'], axis=1).stack())

In [32]:
tbl_daily = pd.DataFrame(daily.set_index(['Date_Confirmed']).rename_axis(['Region'], axis=1).stack())

In [33]:
tbl = pd.merge(tbl_total, tbl_daily, how='inner', on=['Date_Confirmed', 'Region'])

In [34]:
tbl = tbl.rename(columns={'0_x': 'total', '0_y': 'daily'})

In [35]:
tbl.tail()

total  daily
Date_Confirmed Region                  
18-04-2020     TLAXCALA     70.0   10.0
               VERACRUZ     68.0    9.0
               YUCATAN     213.0   13.0
               ZACATECAS    28.0    5.0
               Mexico_SS  6371.0  622.0

In [52]:
# Right format for plotly (long, instead of wide)
# https://plotly.com/python/animations/#using-a-slider-and-buttons
# https://raw.githubusercontent.com/plotly/datasets/master/gapminderDataFiveYear.csv
plot_data = (tbl.reset_index()).to_csv('output_data/time_series/total-daily-tbl.csv', index=False)